# n2c2

## Checking the annotations file

In [62]:
n2c2_ann = [line.strip().split("\t") for line in open("Downloads/training_20180910/100035.ann", "r")]

In [63]:
for i in n2c2_ann:
    sent=i[1].split()
    try:
        s=int(sent[-1])
        if len(sent) > 3:
            print(sent)
    except:
        continue

['Frequency', '15066', '15076;15077', '15095']
['Frequency', '15385', '15390;15391', '15398']
['Form', '15460', '15473;15474', '15483']
['Dosage', '15598', '15601;15602', '15605']
['Frequency', '15955', '15957;15958', '15983']
['Frequency', '16314', '16336;16337', '16343']
['Frequency', '16620', '16632;16633', '16649']
['Frequency', '16839', '16850;16851', '16860']
['Reason', '9314', '9323;9324', '9331']
['Frequency', '15164', '15176;15177', '15184']
['Frequency', '15314', '15328;15329', '15333']
['Frequency', '14964', '14978;14979', '14993']
['Frequency', '16404', '16416;16417', '16423']


In [64]:
n2c2_result = []
for item in n2c2_ann:
    words = item[1].split()
    try:
    # check if the second last word is a number
        if words[-2].isdigit() and words[-1].isdigit():
            start, end = map(int, [words[-2], words[-1]])
            # check if the length of the string in the 3rd position is equal to the difference between the two numbers
            if end - start == len(item[2]):
                n2c2_result.append([item[0], words[0], f"{start}:{end}", item[2]])
        
    except:
        continue

print(n2c2_result[:10])

[['T1', 'Reason', '10179:10197', 'recurrent seizures'], ['T3', 'Drug', '10227:10233', 'ativan'], ['T5', 'Route', '10240:10242', 'IM'], ['T6', 'Drug', '10455:10465', 'Topiramate'], ['T7', 'Strength', '10466:10470', '25mg'], ['T8', 'Route', '10471:10473', 'PO'], ['T9', 'Frequency', '10474:10477', 'BID'], ['T10', 'Duration', '10495:10497', 'PM'], ['T11', 'Strength', '10515:10519', '50mg'], ['T12', 'Route', '10520:10522', 'po']]


## Merging word annotations and text

In [65]:
def load_text(file_path):
    with open(file_path, "r") as file:
        return file.read()

def load_annotations(file_path):
    annotations = []
    with open(file_path, "r") as file:
        for line in file:
            items = line.strip().split("\t")
            try:
                annotations.append([items[0], items[1], items[2].split()])
            except:
                continue
    return annotations

def annotated_text(text, annotations):
    positions = []
    for annotation in annotations:
        if annotation[0].startswith("T"):
            try:
                start, end = map(int, annotation[1].split()[1:])
                positions.append((start, end, annotation[1].split(" ")[0], annotation[2]))
            except:
                start, end = int(annotation[1].split()[1]), int(annotation[1].split()[-1])
                positions.append((start, end, annotation[1].split(" ")[0], annotation[2]))

    # sort the positions list by start index
    positions.sort(key=lambda x: x[0])

    annotated_text = ""
    start = 0
    for position in positions:
        annotated_text += text[start:position[0]]
        annotated_text += f"<{position[2]}> {str(position[3])} </{position[2]}> "
        start = position[1]
    annotated_text += text[start:]
    return annotated_text


In [66]:
def bart_to_ner(text):
    tokens = text.split()
    ner_tags = []
    entity_start = False
    current_entity = ""
    current_tag = 0
    
    for token in tokens:
        if "<" in token and ">" in token:
            if "/" in token:
                entity_start = False
                ner_tags.append((current_entity, current_tag))
                current_entity = ""
                current_tag = 0
            else:
                entity_start = True
                current_tag = token.split("<")[1].split(">")[0]
        elif entity_start:
            current_entity += token + " "
        else:
            ner_tags.append((token, 0))
    ner_taggers=[(x[0].replace("['", "").replace("']", "").strip(), x[1]) for x in ner_tags]        
    return ner_taggers

## Turning annotated text into NER annotations

In [67]:
import re

ner_tags_in_txt =["Drug", "Strength", "Form", "Frequency", "Route", "Dosage", "Reason", "ADE", "Duration"]

def nerring_da_txt(ann_txt):
    
    nerred_txt = []

    for l, i in enumerate(ann_txt):
        if i[1] in ner_tags_in_txt:
            aa=re.findall(r'\b\w+\b', i[0])
            for h, j in enumerate(aa):
                if h==0:
                    nerred_txt.append((j, "B-"+i[1]))
                else:
                    nerred_txt.append((j, "I-"+i[1]))
        #elif i[1] == "ADE":
         #   aa=re.findall(r'\b\w+\b', i[0])
          #  for h, j in enumerate(aa):
           ##     if h==0:
             #       nerred_txt.append((j, "B-"+i[1]))
              #  else:
               #     nerred_txt.append((j, "I-"+i[1]))
        else:
            aa=re.findall(r'\b\w+\b', i[0])
            for j in aa:
                nerred_txt.append((j, 0))
                
    return nerred_txt


In [68]:
text = load_text("Downloads/training_20180910/114144.txt")
annotations = load_annotations("Downloads/training_20180910/114144.ann")
txt = annotated_text(text, annotations)
almost = bart_to_ner(txt)
end_try= nerring_da_txt(almost)

## ALL the files

In [69]:
import os

dir_path = "Downloads/training_20180910/"

filenames = os.listdir(dir_path)

ids = [int(f.split(".")[0]) for f in filenames if f.endswith(".txt")]

ids[:3]

[110727, 101372, 106384]

In [71]:
ner_texts_eng=[]
for nmbr in ids:
    try:
        text = load_text(f"{dir_path}{nmbr}.txt")
        annotations = load_annotations(f"{dir_path}{nmbr}.ann")
        txt = annotated_text(text, annotations)
        ner_txt = bart_to_ner(txt)
        nerred_txt = nerring_da_txt(ner_txt)

        ner_texts_eng.append(nerred_txt)
    except:
        try:
            text = load_text(f"{dir_path}{nmbr}.txt")
            print(nmbr)
        except:
            continue

In [72]:
import csv

with open('output.tsv', 'w', newline='', encoding='utf-8') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    for i, sublist in enumerate(ner_texts_eng):
        for j, (word, label) in enumerate(sublist):
            if label == 0:
                writer.writerow([word, 0])
            else:
                writer.writerow([word, label])
            if j == len(sublist) - 1 and i != len(nerred_txt) - 1:
                writer.writerow([])


In [73]:
len(ner_texts_eng[0])

3557

# Estonian stuff

In [74]:
import pandas as pd

In [75]:
tekst= pd.read_csv("Downloads/training_data_AD_and_AP_1st_and_2nd.csv")

In [78]:
import re

def NER_annotate(text):
    adr_pattern = r'<adr> (\w+) <adr>'
    adr_pattern_2 = r'< adr > (\w+) < adr >'
    
    adr_pattern = r'<( adr )>'
    adr_pattern_2 = r'<(adr)>'
    
    drg_pattern = r'<( drg )>'
    drg_pattern_2 = r'<(drg)>'
    
    annot_pattern = r'<(\w+)>'
    annot_pattern_2 = r'< (\w+) >'

    adr_matches = re.findall(adr_pattern, text)
    adr_matches_2 = re.findall(adr_pattern_2, text)
    
    adr_annot_matches =[i.strip() for i in re.findall(adr_pattern, text)]
    adr_annot_matches_2 = re.findall(adr_pattern_2, text)
    
    drg_annot_matches =[i.strip() for i in re.findall(drg_pattern, text)]
    drg_annot_matches_2 = re.findall(drg_pattern_2, text)
    
    annot_matches = re.findall(annot_pattern, text)
    annot_matches_2 = re.findall(annot_pattern_2, text)
    
    words = re.findall(r'\b\w+\b', text)

    adr_in_annotation = False  # Flag to keep track of whether currently inside an annotation
    drg_in_annotation = False
    the_first = True
    ner_annotated_text = []

    for word in words:
        if adr_in_annotation:
            if word in adr_annot_matches or word in adr_annot_matches_2:
                adr_in_annotation = not adr_in_annotation
                the_first= not the_first
                continue
            elif word in annot_matches or word in annot_matches_2:
                continue 
            elif the_first:
                ner_annotated_text.append((word, 'B-ADE'))
                the_first = not the_first
            else:
                ner_annotated_text.append((word, 'I-ADE'))
        elif drg_in_annotation:
            if word in drg_annot_matches or word in drg_annot_matches_2:
                drg_in_annotation = not drg_in_annotation
                the_first= not the_first
                continue
            elif word in annot_matches or word in annot_matches_2:
                continue 
            elif the_first:
                ner_annotated_text.append((word, 'B-Drug'))
                the_first = not the_first
            else:
                ner_annotated_text.append((word, 'I-Drug'))
        else:
            if word in adr_annot_matches or word in adr_annot_matches_2:
                adr_in_annotation = not adr_in_annotation
                continue
            if word in drg_annot_matches or word in drg_annot_matches_2:
                drg_in_annotation = not drg_in_annotation
                continue
            elif word in annot_matches or word in annot_matches_2:
                continue      
            else:
                ner_annotated_text.append((word, 0))

    return ner_annotated_text


In [79]:
NER_annotate(tekst.iloc[3]["verified"])[:20]

[('CLS', 0),
 ('es', 0),
 ('muutus', 0),
 ('jarjest', 0),
 ('tundlikumaks', 0),
 ('valistele', 0),
 ('sundmustele', 0),
 ('toostressile', 0),
 ('seetottu', 0),
 ('alates', 0),
 ('oktoobrist', 0),
 ('uues', 0),
 ('ti', 'B-Drug'),
 ('kvetiapiin', 'I-Drug'),
 ('tostetud', 0),
 ('pt', 0),
 ('olnud', 0),
 ('tvl', 0),
 ('il', 0),
 ('et', 0)]

In [464]:
tekstid= [NER_annotate(tekst.iloc[i]["verified"]) for i in range(len(tekst))][1:]

In [485]:
for s, i in enumerate(tekstid[700:1800]):
    for l, k in enumerate(i):
        if k[1] =="I-Drug" :
            print(s, i[l])

8 ('fluanxol', 'I-Drug')
14 ('risperidon', 'I-Drug')
15 ('lorasepaam', 'I-Drug')
24 ('risperidoon', 'I-Drug')
44 ('bromazepam', 'I-Drug')
62 ('haloperidool', 'I-Drug')
99 ('lorasepaam', 'I-Drug')
103 ('aripiprasool', 'I-Drug')
106 ('abilify', 'I-Drug')
113 ('haloperidol', 'I-Drug')
120 ('alprazolam', 'I-Drug')
142 ('diasepaam', 'I-Drug')
161 ('diasepaam', 'I-Drug')
168 ('risperidoon', 'I-Drug')
412 ('sertraliin', 'I-Drug')
413 ('sertraliin', 'I-Drug')
424 ('Escitalopram', 'I-Drug')
429 ('estsitalopraam', 'I-Drug')
750 ('Sertralin', 'I-Drug')
751 ('Sertralin', 'I-Drug')
767 ('sertraliin', 'I-Drug')
812 ('i', 'I-Drug')
842 ('Sertralini', 'I-Drug')
843 ('Sertralini', 'I-Drug')
844 ('Sertralini', 'I-Drug')
845 ('Sertralini', 'I-Drug')
921 ('venlafaxini', 'I-Drug')
922 ('venlafaxini', 'I-Drug')
925 ('estsitaloprami', 'I-Drug')
950 ('escitalopram', 'I-Drug')
951 ('escitalopram', 'I-Drug')
984 ('Valdoxan', 'I-Drug')
985 ('Valdoxan', 'I-Drug')
986 ('Valdoxan', 'I-Drug')
991 ('Escitalopram', 'I

In [486]:
with open('est_output.tsv', 'w') as f:
    for sublist in tekstid:
        for tuple in sublist:
            f.write('\t'.join(map(str, tuple)) + '\n')
        f.write('\n')